In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

path = '/content/gdrive/My Drive/activity-classification/'

Mounted at /content/gdrive/


In [ ]:
file_names = []

for root, dirs, files in os.walk(str(path + 'data/')):
    for file in files:
        if not file.startswith('.'):
            file_names.append(os.path.join(root[-6:], file))

print(file_names[0])

a03/p2/s04.txt


In [ ]:
len(file_names)

9120

In [ ]:
column_names = []

for position in ['t', 'ra', 'la', 'rl', 'll']:
    for sensor in ['acc', 'gyro', 'mag']:
        for plane in ['x', 'y', 'z']:
            column_names.append(position + '_' + plane + sensor)

In [ ]:
# read in the first dataset
df = pd.read_csv(str(path + 'data/' + file_names[0]), names=column_names, header=None, dtype='float32')

# build the starting dataframe
df['activity_code'] = file_names[0][:3]
df['person'] = file_names[0][5:6]
df['segment'] = file_names[0][8:10]

df.head()

,t_xacc,t_yacc,t_zacc,t_xgyro,t_ygyro,t_zgyro,t_xmag,t_ymag,t_zmag,ra_xacc,...,ll_zacc,ll_xgyro,ll_ygyro,ll_zgyro,ll_xmag,ll_ymag,ll_zmag,activity_code,person,segment
0,-1.6313,0.42783,9.6477,-0.057136,-0.038351,0.026476,0.45683,-0.051127,-0.78315,0.126830,...,-1.4745,-0.09809,-0.100020,-0.007653,-0.49465,0.77763,0.25364,a03,2,04
1,-1.5270,0.42699,9.6485,-0.052418,-0.012576,0.009966,0.45668,-0.049241,-0.78585,0.106760,...,-1.9034,-0.21934,-0.145700,0.011416,-0.49458,0.77494,0.25955,a03,2,04
2,-1.5343,0.48619,9.6787,-0.118570,-0.002875,0.030813,0.45619,-0.049516,-0.78507,0.087413,...,-2.2311,-0.32198,-0.168840,-0.004813,-0.49413,0.77326,0.27264,a03,2,04
3,-1.4973,0.40456,9.6411,-0.065907,0.027309,0.037138,0.45626,-0.047415,-0.78615,0.110190,...,-2.3645,-0.25921,-0.135090,-0.030112,-0.49179,0.76869,0.28545,a03,2,04
4,-1.4980,0.27095,9.7074,-0.052975,0.059181,0.012394,0.45613,-0.046251,-0.78603,0.132050,...,-2.5285,-0.12898,-0.085426,-0.036430,-0.49075,0.76640,0.29191,a03,2,04


In [ ]:
def add_dataset(file, path=path):
    temp = pd.read_csv(str(path + file), names=column_names, header=None, dtype='float32')
    temp['activity_code'] = file[:3]
    temp['person'] = file[5:6]
    temp['segment'] = file[8:10]
    return temp

In [ ]:
# add all files to one dataset and label
for file in file_names[1:]:
    df = df.append(add_dataset(file))
    
# [add_dataset(file) for file in file_names]

FileNotFoundError: ignored

In [ ]:
len(df)

In [ ]:
df.to_pickle(str(path + 'my-data/raw_import.pkl'))

# Formatting data

Read data, reformat activity codes and build a time column, remove null rows

In [ ]:
df = pd.read_pickle(str(path + '/my-data/raw_import.pkl'))

# format activity_codes and index at zero
df['activity_code'] = df['activity_code'].str.replace(r'a', '')
df['activity_code'] = df['activity_code'].astype('int32')
df['activity'] = df['activity_code']-1
df.drop('activity_code', axis=1, inplace=True)

# set dtypes


# use index labels to label partitions of each segment and convert to seconds
df.reset_index(inplace=True)
df.rename(columns = {'index':'partition'}, inplace=True)
df['segment'] = df['segment'].astype('int32')
df['partition'] = df['partition'].astype('int32')
df['seconds'] = ((df['segment']-1)*5)+(df['partition']*0.04)

# set index
df = df.set_index(['activity', 'person', 'seconds'])
df.sort_index(inplace=True)
df = df.drop(['segment', 'partition'], axis=1)
df = df.astype('float32')

# remove zero rows
zero_rows = df[(df.iloc[:,1:46] == 0).all(axis=1)]
df = df.drop(zero_rows.index)
print('All rows counted: ' + str(len(df)+len(zero_rows)==1140000) + '    Zero rows removed: ' + str(len(zero_rows)))

# store resulting df
df = df.reset_index()
df.to_pickle(str(path+'/my-data/df_original.pkl'))